Feature Engineering first: generating more features based on all the features we have now

In [1]:
from sklearn import preprocessing
import pandas as pd

In [24]:
'''
data: the raw input data we have 
'''
def Generate_Features(data):
    data['Count'] = data['ScanCount']
    data['Count'][data['ScanCount']<0] = 0  # filter the negative values
    data['FinelineNumber'].fillna(value = 9999, inplace = True) # replace the na values with 9999
    data['Upc'].fillna(value = -100, inplace = True)# replace the na values with -100
    
    # encoding the department description label by sklearn
    le = preprocessing.LabelEncoder()
    # encode and transform the department description label
    data['DepartmentDescription'] = le.fit_transform(list(data['DepartmentDescription']))
    data['Weekday'] = preprocessing.LabelEncoder().fit_transform(list(data['Weekday']))
    
    #===================== Missing Value Indicators ======================================#
    # null value exist in Department Description, encoded as 67
    data1 = data[data['DepartmentDescription'] == 67]
    # the number of non na observations of each visit number
    data1 = data[data['DepartmentDescription']==67]
    data1 = data1.groupby(['VisitNumber'], as_index=False)['Count'].count()
    data1.rename(columns={'Count': 'Count_Null'}, inplace=True) 
    data = data.merge(data1, how='left', on=['VisitNumber'], copy=True) 
    data['Count_Null'].fillna(value=0, inplace=True) 
    data['Count_Null'][data['Count_Null']>0] = 1 # 把count 换成1

    data1 = data[data['ScanCount']<0]
    data1 = data1.groupby(['VisitNumber'], as_index=False)['Count'].count()
    data1.rename(columns={'Count': 'ScanCount_Neg'}, inplace=True) 
    data = data.merge(data1, how='left', on=['VisitNumber'], copy=True) 
    data['ScanCount_Neg'].fillna(value=0, inplace=True)
    data['ScanCount_Neg'][data['ScanCount_Neg']>0] = 1
    
    data1 = data[data['FinelineNumber']==9999]
    data1 = data1.groupby(['VisitNumber'], as_index=False)['Count'].count()
    data1.rename(columns={'Count': 'FinelineNumber_Missing'}, inplace=True) 
    data = data.merge(data1, how='left', on=['VisitNumber'], copy=True) 
    data['FinelineNumber_Missing'].fillna(value=0, inplace=True)
    data['FinelineNumber_Missing'][data['FinelineNumber_Missing']>0] = 1

    data1 = data.groupby(['VisitNumber',  'FinelineNumber'], as_index=False)['Count'].count()
    data1 = data1.groupby(['VisitNumber'], as_index=False)['Count'].count()
    data1.rename(columns={'Count': 'N_Fineline'}, inplace=True) 
    data = data.merge(data1, how='left', on=['VisitNumber'], copy=True)
         
    data1 = data.groupby(['VisitNumber',  'Upc'], as_index=False)['Count'].count()
    data1 = data1.groupby(['VisitNumber'], as_index=False)['Count'].count()
    data1.rename(columns={'Count': 'N_Upc'}, inplace=True) 
    data = data.merge( data1, how='left', on=['VisitNumber'], copy=True) 

    data1 = data.groupby(['VisitNumber', 'DepartmentDescription'], as_index=False)['Count'].count()
    data1 = data1.groupby(['VisitNumber'], as_index=False)['Count'].count()
    data1.rename(columns={'Count': 'N_Dep'}, inplace=True) 
    data = data.merge( data1, how='left', on=['VisitNumber'], copy=True)

    
    # group data for new features:
    # 1. visit number and departmant description
    # the scan counts for each visitnumber and department combination
    temp1 = data.groupby(['VisitNumber', 'DepartmentDescription'], as_index=False)['ScanCount'].sum()
    temp11 = temp1.groupby(['VisitNumber'], as_index=False)['ScanCount'].min()
    temp12 = temp1.groupby(['VisitNumber'], as_index=False)['ScanCount'].max()
    temp13 = temp1.groupby(['VisitNumber'], as_index=False)['ScanCount'].mean()
    temp11.rename(columns={'ScanCount': 'Min_Count'}, inplace=True)
    temp12.rename(columns={'ScanCount': 'Max_Count'}, inplace=True)
    temp13.rename(columns={'ScanCount': 'Mean_Count'}, inplace=True)
    # left join to the dataset
    data = data.merge(temp11, how='left', on=['VisitNumber'], copy=True)
    data = data.merge(temp12, how='left', on=['VisitNumber'], copy=True)
    data = data.merge(temp13, how='left', on=['VisitNumber'], copy=True)         
    data['Range'] = data['Max_Count'] - data['Min_Count'] 
    
    data['Ratio_F_D'] = data['N_Fineline']/data['N_Dep']
    data['Ratio_U_D'] = data['N_Upc']/data['N_Dep'] 
    data['mean_to_min'] = data['Mean_Count']/data['Min_Count']
    data['mean_to_min'][data['Min_Count']==0] = 0
    data['max_to_mean'] = data['Max_Count']/data['Mean_Count']
    data['max_to_mean'][data['Mean_Count']==0] = 0    
    
    return data

In [7]:
path = 'C:\Users\shuyi\Documents\StudyResource\Kaggle\\'
train_file = "train.csv"
train_data = pd.read_csv(path + train_file)

In [25]:
train_data_step2 = Generate_Features(train_data)

c:\users\shuyi\anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\shuyi\anaconda2\lib\site-packages\ipykernel\__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\shuyi\anaconda2\lib\site-packages\ipykernel\__main__.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\shuyi\anaconda2\lib\site-packages\ipykernel\__main__.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [26]:
train_data_step2[1:10]

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Count,Count_Null,ScanCount_Neg,...,N_Upc,N_Dep,Min_Count,Max_Count,Mean_Count,Range,Ratio_F_D,Ratio_U_D,mean_to_min,max_to_mean
1,30,7,0,6.053882e+10,1,62,8931.0,1,0.0,0.0,...,2,2,1,1,1.0,0,1.000000,1.0,1.0,1.0
2,30,7,0,7.410811e+09,1,50,4504.0,1,0.0,0.0,...,2,2,1,1,1.0,0,1.000000,1.0,1.0,1.0
3,26,8,0,2.238404e+09,2,49,3565.0,2,0.0,1.0,...,21,7,1,18,4.0,17,2.428571,3.0,4.0,4.5
4,26,8,0,2.006614e+09,2,49,1017.0,2,0.0,1.0,...,21,7,1,18,4.0,17,2.428571,3.0,4.0,4.5
5,26,8,0,2.006619e+09,2,49,1017.0,2,0.0,1.0,...,21,7,1,18,4.0,17,2.428571,3.0,4.0,4.5
6,26,8,0,2.006614e+09,1,49,1017.0,1,0.0,1.0,...,21,7,1,18,4.0,17,2.428571,3.0,4.0,4.5
7,26,8,0,7.004803e+09,1,49,2802.0,1,0.0,1.0,...,21,7,1,18,4.0,17,2.428571,3.0,4.0,4.5
8,26,8,0,2.238495e+09,1,49,4501.0,1,0.0,1.0,...,21,7,1,18,4.0,17,2.428571,3.0,4.0,4.5
9,26,8,0,2.238400e+09,-1,49,3565.0,0,0.0,1.0,...,21,7,1,18,4.0,17,2.428571,3.0,4.0,4.5


In [27]:
# save the data 
train_data_step2.to_csv(path + "step2.csv")